In [1]:
# 이걸 잠시 하다보니 다른 생각이 든다..
# 필터링을 위한 메타데이터..가 존재해야하나? 벡터 db에서 필터링을 해야하나?
# 단순히 연산이 작아보이고 쉬워보여서.. 대충 메타데이터에 넣고, 벡터db에서 필터링 하게 한 게 잘못된 건 아닐까?
# 그냥 이러한 필터링 자체는 최대한 rdb 단에서 수행하고, 메타데이터와 벡터db의 사용을 최적화 하는 게 더 좋지 않을까?
# => 그렇게 하자. 처음부터 다시하자.

In [9]:
def populate_code_tables(cursor):
    """
    엑셀 파일의 각 시트를 읽어 코드 테이블에 데이터를 삽입합니다.
    """
    print("🚀 코드 테이블 데이터 이관 시작...")

    try:
        # 테이블과 시트 이름, 그리고 해당 시트 내의 컬럼명을 매핑합니다.
        # 시트 이름은 실제 엑셀 파일의 시트 이름과 정확히 일치해야 합니다.
        code_sheets_map = {
            'major_codes': ('코드정보', 'plcyMajorCd', '코드', '코드내용'),
            'job_status_codes': ('코드정보', 'jobCd', '코드', '코드내용'),
            'education_level_codes': ('코드정보', 'schoolCd', '코드', '코드내용'),
            'specialization_codes': ('코드정보', 'sBizCd', '코드', '코드내용'),
            'category_codes': ('정책대분류', None, '번호', '정책대분류명(lclsfNm)'),
            'subcategory_codes': ('정책중분류', None, '번호', '정책중분류명(mclsfNm)'),
            'keywords': ('정책키워드', None, None, '정책키워드명(plcyKywdNm)')
        }
        
        for table_name, (sheet_name, filter_col, code_col, name_col) in code_sheets_map.items():
            print(f"  - 테이블 '{table_name}' 작업 중 (시트: '{sheet_name}')...")
            # read_excel을 사용하여 특정 시트를 읽어옵니다.
            df = pd.read_excel(excel_file_path, sheet_name=sheet_name)
            
            # '코드정보' 시트의 경우, 특정 분류(jobCd 등)로 필터링
            if filter_col:
                df = df[df['분류'] == filter_col].copy()
            
            insert_sql = ""
            # 각 테이블 구조에 맞게 데이터 준비 및 INSERT
            if table_name == 'keywords':
                insert_sql = f"INSERT INTO {table_name} (name) VALUES (%s)"
                data_to_insert = [(row[name_col],) for index, row in df.iterrows() if pd.notna(row[name_col])]
            else:
                insert_sql = f"INSERT INTO {table_name} (code, name) VALUES (%s, %s)"
                data_to_insert = [(str(row[code_col]), row[name_col]) for index, row in df.iterrows() if pd.notna(row[code_col]) and pd.notna(row[name_col])]
            
            cursor.executemany(insert_sql, data_to_insert)
            print(f"    ✅ {cursor.rowcount}개 행 삽입 완료.")

    except FileNotFoundError:
        print(f"🚨 오류: 엑셀 파일('{excel_file_path}')을 찾을 수 없습니다. 파일 이름과 경로를 확인하세요.")
        raise
    except Exception as e:
        print(f"🚨 오류: 코드 테이블 이관 중 문제 발생 - {e}")
        raise


In [10]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': '1234',
    'database': 'toyprj4'
}

# 엑셀 파일 이름 (사용자님이 변경하신 이름)
excel_file_path = './code_table.xlsx'

In [14]:
connection = None
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
print("✅ 데이터베이스 연결 성공!")

populate_code_tables(cursor)

✅ 데이터베이스 연결 성공!
🚀 코드 테이블 데이터 이관 시작...
  - 테이블 'major_codes' 작업 중 (시트: '코드정보')...
    ✅ 1개 행 삽입 완료.
  - 테이블 'job_status_codes' 작업 중 (시트: '코드정보')...
    ✅ 1개 행 삽입 완료.
  - 테이블 'education_level_codes' 작업 중 (시트: '코드정보')...
    ✅ 1개 행 삽입 완료.
  - 테이블 'specialization_codes' 작업 중 (시트: '코드정보')...
    ✅ 1개 행 삽입 완료.
  - 테이블 'category_codes' 작업 중 (시트: '정책대분류')...
    ✅ 5개 행 삽입 완료.
  - 테이블 'subcategory_codes' 작업 중 (시트: '정책중분류')...
    ✅ 17개 행 삽입 완료.
  - 테이블 'keywords' 작업 중 (시트: '정책키워드')...
    ✅ 17개 행 삽입 완료.


In [16]:
connection.commit()
print("\n🎉 모든 작업이 성공적으로 완료되었습니다.")


🎉 모든 작업이 성공적으로 완료되었습니다.


In [13]:
# connection.rollback()

In [9]:
# 1. 데이터 불러오기

import pandas as pd
import numpy as np

# 데이터가 담긴 CSV 파일 경로
csv_file_path = './policy_data.csv' # 실제 파일 이름으로 변경해주세요.

try:
    # CSV 파일을 DataFrame으로 읽어옵니다.
    # 한글 깨짐 방지를 위해 encoding='utf-8'을 사용합니다.
    df_raw = pd.read_csv(csv_file_path, encoding='utf-8')
    
    # 데이터 로딩 성공 확인
    print("✅ 1단계 성공: 데이터 로딩 완료!")
    print(f"전체 정책 데이터 수: {len(df_raw)}개")
    
    # 데이터 구조 확인을 위해 상위 3개 행을 출력합니다.
    print("\n--- 원본 데이터 샘플 ---")
    print(df_raw.head(3))
    
except FileNotFoundError:
    print(f"❌ 오류: '{csv_file_path}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
except Exception as e:
    print(f"❌ 오류: 파일을 읽는 중 문제가 발생했습니다. (오류: {e})")

✅ 1단계 성공: 데이터 로딩 완료!
전체 정책 데이터 수: 3861개

--- 원본 데이터 샘플 ---
                 plcyNo  bscPlanCycl  bscPlanPlcyWayNo  bscPlanFcsAsmtNo  \
0  20250717005400211359            1                 4                16   
1  20250717005400211358            1                 4                16   
2  20250717005400211357            1                 4                16   

   bscPlanAsmtNo  pvsnInstGroupCd  plcyPvsnMthdCd  plcyAprvSttsCd  \
0             41            54002         42002.0           44002   
1             43            54002         42004.0           44002   
2             43            54002         42004.0           44002   

                   plcyNm           plcyKywdNm  ... rgtrHghrkInstCd  \
0        웹툰캠퍼스 운영 및 인력 양성  교육지원,인턴,맞춤형상담서비스,벤처  ...         6270000   
1               학예사 인턴 운영                   인턴  ...         6540000   
2  전북청년 2025 (전북청년 미술 운영)                  보조금  ...         6540000   

  rgtrHghrkInstCdNm                                              zipCd  \


In [12]:
# 2. 데이터 정제 및 가공
# 숫자 형 변환 : 나이, 소득 등 숫자여야 하는 열들의 데이터 타입을 숫자로 확실하게 변환, 숫자로 변환 불가한 경우 NaN 처리
# 날짜 형 변환 : yymmdd 형식의 날짜륻릉ㄹ 실제 날짜 형으로 변환하고, 구간 정보의 경우 분리.

# --- 2.1. 컬럼 선택 및 이름 변경 (수정) ---
column_mapping = {
    'plcyNo': 'policy_id',
    'plcyNm': 'policy_name',
    'plcyExplnCn': 'policy_summary',
    'refUrlAddr1': 'source_url',
    'sprtTrgtMinAge': 'min_age',
    'sprtTrgtMaxAge': 'max_age',
    'earnMinAmt': 'income_min',
    'earnMaxAmt': 'income_max',
    'bizPrdBgngYmd': 'biz_start_date',
    'bizPrdEndYmd': 'biz_end_date',
    'aplyYmd': 'aply_period',
    'mrgSttsCd': 'marriage_status',
    'aplyPrdSeCd': 'application_status', 
    'jobCd': 'job_status_codes',
    'schoolCd': 'education_level_codes',
    'plcyMajorCd': 'major_codes',
    'plcyKywdNm': 'keywords',
    'zipCd': 'region_codes',
    'lclsfNm': 'category_names',
    'mclsfNm': 'subcategory_names'
}

df_selected = df_raw[list(column_mapping.keys())].copy()
df_processed = df_selected.rename(columns=column_mapping)

# --- 2.2. 숫자 형식 변환 (변경 없음) ---
numeric_cols = ['min_age', 'max_age', 'income_min', 'income_max']
for col in numeric_cols:
    df_processed[col] = pd.to_numeric(df_processed[col], errors='coerce')
    df_processed[col] = df_processed[col].replace(0, np.nan)

# --- 2.3. 날짜 형식 변환 (변경 없음) ---
def to_datetime_safe(date_str):
    if pd.isna(date_str) or not isinstance(date_str, str) or not date_str.strip():
        return pd.NaT
    return pd.to_datetime(date_str, format='%Y%m%d', errors='coerce')

def parse_apply_period(period_str):
    if pd.isna(period_str) or '~' not in str(period_str):
        return pd.NaT, pd.NaT
    try:
        start_str, end_str = [s.strip() for s in period_str.split('~')]
        return to_datetime_safe(start_str), to_datetime_safe(end_str)
    except ValueError:
        return pd.NaT, pd.NaT

df_processed['biz_start_date'] = df_processed['biz_start_date'].apply(to_datetime_safe)
df_processed['biz_end_date'] = df_processed['biz_end_date'].apply(to_datetime_safe)
df_processed[['aply_start_date', 'aply_end_date']] = df_processed['aply_period'].apply(
    lambda x: pd.Series(parse_apply_period(x))
)
df_processed = df_processed.drop(columns=['aply_period'])

print("✅ 수정된 2단계 성공: `application_status` 포함하여 정제 완료!")
print(df_processed[['policy_id', 'application_status']].head())

✅ 수정된 2단계 성공: `application_status` 포함하여 정제 완료!
              policy_id  application_status
0  20250717005400211359               57001
1  20250717005400211358               57002
2  20250717005400211357               57001
3  20250717005400211356               57002
4  20250717005400211355               57002


In [13]:
# 3. policies 테이블용 데이터 프레임 생성

# 1. 'policies' 테이블 컬럼 목록 정의 (수정)
policy_table_cols = [
    'policy_id',
    'policy_name',
    'policy_summary',
    'source_url',
    'min_age',
    'max_age',
    'income_min',
    'income_max',
    'biz_start_date',
    'biz_end_date',
    'aply_start_date',
    'aply_end_date',
    'marriage_status',
    'application_status' 
]

# 2. df_processed에서 해당 컬럼들만 선택하여 최종 policies_df 생성
policies_df = df_processed[policy_table_cols].copy()

# 3. 결과 확인
print("\n✅ 수정된 3단계 성공: `application_status` 포함하여 `policies_df` 생성 완료!")
print("\n--- policies_df 정보 ---")
policies_df.info()


✅ 수정된 3단계 성공: `application_status` 포함하여 `policies_df` 생성 완료!

--- policies_df 정보 ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3861 entries, 0 to 3860
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   policy_id           3861 non-null   object        
 1   policy_name         3861 non-null   object        
 2   policy_summary      3861 non-null   object        
 3   source_url          2660 non-null   object        
 4   min_age             2436 non-null   float64       
 5   max_age             2454 non-null   float64       
 6   income_min          1 non-null      float64       
 7   income_max          7 non-null      float64       
 8   biz_start_date      795 non-null    datetime64[ns]
 9   biz_end_date        796 non-null    datetime64[ns]
 10  aply_start_date     2475 non-null   datetime64[ns]
 11  aply_end_date       2475 non-null   datetime64[ns]
 12  marriage_status   

In [14]:
# 순서 재정렬(DB 스키마 순서와 동일하게)
# (선택 사항) DB 스키마 순서와 동일하게 컬럼 순서 재정렬
db_schema_order = [
    'policy_id', 'min_age', 'max_age', 'income_min', 'income_max',
    'biz_start_date', 'biz_end_date', 'aply_start_date', 'aply_end_date',
    'application_status', 'marriage_status', 'policy_name', 
    'policy_summary', 'source_url'
]
policies_df = policies_df[db_schema_order]

# 재정렬된 순서 확인
print(policies_df.head())

              policy_id  min_age  max_age  income_min  income_max  \
0  20250717005400211359      NaN      NaN         NaN         NaN   
1  20250717005400211358     18.0     39.0         NaN         NaN   
2  20250717005400211357     18.0     39.0         NaN         NaN   
3  20250717005400211356      NaN      NaN         NaN         NaN   
4  20250717005400211355     19.0     39.0         NaN         NaN   

  biz_start_date biz_end_date aply_start_date aply_end_date  \
0     2025-05-01   2025-12-31      2025-05-01    2025-05-30   
1     2025-01-01   2025-12-31             NaT           NaT   
2     2025-01-01   2025-12-31      2024-10-07    2024-10-18   
3            NaT          NaT             NaT           NaT   
4     2025-03-18   2025-12-12             NaT           NaT   

   application_status  marriage_status             policy_name  \
0               57001          55003.0        웹툰캠퍼스 운영 및 인력 양성   
1               57002          55003.0               학예사 인턴 운영   
2       

In [18]:
df_processed.info()

# df_processed에서 'keywords' 열이 비어있지 않은 행의 개수를 세어봅니다.
policies_with_keywords = df_processed['keywords'].notna().sum()

print(f"전체 {len(df_processed)}개 정책 중, 키워드가 할당된 정책의 수: {policies_with_keywords}개")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3861 entries, 0 to 3860
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   policy_id              3861 non-null   object        
 1   policy_name            3861 non-null   object        
 2   policy_summary         3861 non-null   object        
 3   source_url             2660 non-null   object        
 4   min_age                2436 non-null   float64       
 5   max_age                2454 non-null   float64       
 6   income_min             1 non-null      float64       
 7   income_max             7 non-null      float64       
 8   biz_start_date         795 non-null    datetime64[ns]
 9   biz_end_date           796 non-null    datetime64[ns]
 10  marriage_status        1062 non-null   float64       
 11  application_status     3861 non-null   int64         
 12  job_status_codes       3861 non-null   object        
 13  edu

In [15]:
# 4. 매핑 테이블용 데이터 프레임 생성


# 1. 매핑 테이블 생성을 위한 범용 함수 정의
def create_mapping_df(df, id_col, value_col, new_value_col_name):
    """
    쉼표로 구분된 값을 가진 컬럼을 분리하여,
    정책 ID와 값이 1:1로 매핑되는 '길쭉한' 데이터프레임을 만듭니다.
    
    :param df: 원본 데이터프레임 (df_processed)
    :param id_col: 기준 ID 컬럼명 ('policy_id')
    :param value_col: 쉼표로 구분된 값이 있는 컬럼명
    :param new_value_col_name: 최종 매핑 테이블의 값 컬럼명
    :return: 매핑 테이블용 데이터프레임
    """
    # 1. 필요한 컬럼만 선택하고, 값이 비어있는 행은 제외
    temp_df = df[[id_col, value_col]].dropna(subset=[value_col])
    
    # 2. 쉼표(,)를 기준으로 값을 분리하여 리스트로 만듦
    temp_df[value_col] = temp_df[value_col].str.split(',')
    
    # 3. .explode() 함수로 리스트의 각 항목을 별도의 행으로 펼침
    mapping_df = temp_df.explode(value_col)
    
    # 4. 각 값의 앞뒤 공백 제거
    mapping_df[value_col] = mapping_df[value_col].str.strip()
    
    # 5. 최종 컬럼 이름 변경 (예: value_col -> keyword_name)
    mapping_df = mapping_df.rename(columns={value_col: new_value_col_name})
    
    # 6. 인덱스 초기화
    mapping_df = mapping_df.reset_index(drop=True)
    
    return mapping_df

# 2. 'keywords' 컬럼을 사용하여 'policy_keywords' 매핑 테이블 생성
policy_keywords_df = create_mapping_df(
    df=df_processed, 
    id_col='policy_id', 
    value_col='keywords', 
    new_value_col_name='keyword_name' # DB 스키마에 맞게 이름 지정
)

# 3. 결과 확인
print("✅ 4-1단계 성공: `policy_keywords_df` 생성 완료!")
print(f"생성된 키워드 매핑 수: {len(policy_keywords_df)}개")

print("\n--- policy_keywords_df 데이터 샘플 ---")
print(policy_keywords_df.head(10))

✅ 4-1단계 성공: `policy_keywords_df` 생성 완료!
생성된 키워드 매핑 수: 1720개

--- policy_keywords_df 데이터 샘플 ---
              policy_id keyword_name
0  20250717005400211359         교육지원
1  20250717005400211359           인턴
2  20250717005400211359     맞춤형상담서비스
3  20250717005400211359           벤처
4  20250717005400211358           인턴
5  20250717005400211357          보조금
6  20250717005400211356      장기미취업청년
7  20250717005400211355         교육지원
8  20250717005400211354           벤처
9  20250717005400211354         중소기업


In [19]:

print("✅ 4-2단계 시작: 나머지 매핑 테이블들을 생성합니다.\n")

# 1. 취업 상태(job_status) 매핑 테이블 생성
policy_job_status_df = create_mapping_df(
    df=df_processed, 
    id_col='policy_id', 
    value_col='job_status_codes', 
    new_value_col_name='job_status_code'
)
print("--- `policy_job_status_df` 생성 완료 ---")
print(policy_job_status_df.head())


# 2. 학력(education_levels) 매핑 테이블 생성
policy_education_levels_df = create_mapping_df(
    df=df_processed, 
    id_col='policy_id', 
    value_col='education_level_codes', 
    new_value_col_name='education_level_code'
)
print("\n--- `policy_education_levels_df` 생성 완료 ---")
print(policy_education_levels_df.head())


# 3. 전공(majors) 매핑 테이블 생성
policy_majors_df = create_mapping_df(
    df=df_processed, 
    id_col='policy_id', 
    value_col='major_codes', 
    new_value_col_name='major_code'
)
print("\n--- `policy_majors_df` 생성 완료 ---")
print(policy_majors_df.head())


# 4. 지역(regions) 매핑 테이블 생성
policy_regions_df = create_mapping_df(
    df=df_processed, 
    id_col='policy_id', 
    value_col='region_codes', 
    new_value_col_name='region_code'
)
print("\n--- `policy_regions_df` 생성 완료 ---")
print(policy_regions_df.head())


# 5. 카테고리(categories) 매핑 테이블 생성
policy_categories_df = create_mapping_df(
    df=df_processed, 
    id_col='policy_id', 
    value_col='category_names', 
    new_value_col_name='category_name'
)
print("\n--- `policy_categories_df` 생성 완료 ---")
print(policy_categories_df.head())


# 6. 서브카테고리(subcategories) 매핑 테이블 생성
policy_subcategories_df = create_mapping_df(
    df=df_processed, 
    id_col='policy_id', 
    value_col='subcategory_names', 
    new_value_col_name='subcategory_name'
)
print("\n--- `policy_subcategories_df` 생성 완료 ---")
print(policy_subcategories_df.head())

print("\n\n✅ 4단계 성공: 모든 매핑 테이블용 데이터프레임 생성 완료!")

✅ 4-2단계 시작: 나머지 매핑 테이블들을 생성합니다.

--- `policy_job_status_df` 생성 완료 ---
              policy_id job_status_code
0  20250717005400211359         0013010
1  20250717005400211358         0013003
2  20250717005400211357         0013010
3  20250717005400211356         0013010
4  20250717005400211355         0013010

--- `policy_education_levels_df` 생성 완료 ---
              policy_id education_level_code
0  20250717005400211359              0049010
1  20250717005400211358              0049007
2  20250717005400211358              0049008
3  20250717005400211358              0049009
4  20250717005400211357              0049010

--- `policy_majors_df` 생성 완료 ---
              policy_id major_code
0  20250717005400211359    0011009
1  20250717005400211358    0011006
2  20250717005400211357    0011009
3  20250717005400211356    0011009
4  20250717005400211355    0011009

--- `policy_regions_df` 생성 완료 ---
              policy_id region_code
0  20250717005400211359       27110
1  20250717005400211359  

In [20]:
# 5. 데이터 적재

from sqlalchemy import create_engine
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- 5.1. DB 연결 설정 (!!!반드시 수정!!!) ---
# MySQL 데이터베이스 연결 정보
# 예: "mysql+pymysql://<user>:<password>@<host>:<port>/<dbname>"
DB_USER = "root"
DB_PASSWORD = "1234"
DB_HOST = "localhost"  # 또는 DB 서버의 IP 주소
DB_PORT = "3306"
DB_NAME = "toyprj4" # 이전에 생성한 데이터베이스 이름

# SQLAlchemy 연결 문자열 생성
DB_URL = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"


# --- 5.2. 데이터베이스 엔진 생성 및 데이터 적재 ---

try:
    # 데이터베이스 엔진 생성
    engine = create_engine(DB_URL)
    logging.info(f"데이터베이스 '{DB_NAME}'에 성공적으로 연결되었습니다.")

    # 적재할 데이터프레임과 DB 테이블 이름을 딕셔너리로 매핑
    # 이전에 생성한 모든 데이터프레임 변수들이 메모리에 있어야 합니다.
    dataframes_to_load = {
        'policies': policies_df,
        'policy_keywords': policy_keywords_df,
        'policy_job_status': policy_job_status_df,
        'policy_education_levels': policy_education_levels_df,
        'policy_majors': policy_majors_df,
        'policy_regions': policy_regions_df,
        'policy_categories': policy_categories_df,
        'policy_subcategories': policy_subcategories_df
    }

    logging.info("데이터 적재를 시작합니다...")

    # 딕셔너리를 순회하며 각 데이터프레임을 해당 테이블에 적재
    for table_name, df in dataframes_to_load.items():
        try:
            # to_sql 함수를 사용하여 데이터를 테이블에 삽입
            df.to_sql(
                name=table_name,       # 데이터를 삽입할 테이블 이름
                con=engine,            # 데이터베이스 연결 엔진
                if_exists='replace',   # 테이블이 이미 있으면 어떻게 할지 결정
                                       # 'replace': 기존 테이블 삭제 후 새로 생성
                                       # 'append': 기존 테이블에 데이터 추가
                                       # 'fail': 오류 발생
                index=False            # DataFrame의 인덱스를 DB에 컬럼으로 추가하지 않음
            )
            logging.info(f"✅ 테이블 '{table_name}'에 {len(df)}개의 행을 성공적으로 적재했습니다.")
        except Exception as e:
            logging.error(f"❌ 테이블 '{table_name}' 적재 중 오류 발생: {e}")

    logging.info("🎉 모든 데이터 적재 작업이 성공적으로 완료되었습니다!")

except Exception as e:
    logging.error(f"❌ 데이터베이스 연결 또는 처리 중 심각한 오류 발생: {e}")
    logging.error("DB 연결 정보를 다시 확인해주세요.")



2025-07-24 12:25:41,384 - INFO - 데이터베이스 'toyprj4'에 성공적으로 연결되었습니다.
2025-07-24 12:25:41,385 - INFO - 데이터 적재를 시작합니다...
2025-07-24 12:25:41,642 - INFO - ✅ 테이블 'policies'에 3861개의 행을 성공적으로 적재했습니다.
2025-07-24 12:25:41,701 - INFO - ✅ 테이블 'policy_keywords'에 1720개의 행을 성공적으로 적재했습니다.
2025-07-24 12:25:41,793 - INFO - ✅ 테이블 'policy_job_status'에 4682개의 행을 성공적으로 적재했습니다.
2025-07-24 12:25:41,870 - INFO - ✅ 테이블 'policy_education_levels'에 4992개의 행을 성공적으로 적재했습니다.
2025-07-24 12:25:41,946 - INFO - ✅ 테이블 'policy_majors'에 4387개의 행을 성공적으로 적재했습니다.
2025-07-24 12:25:43,572 - INFO - ✅ 테이블 'policy_regions'에 160488개의 행을 성공적으로 적재했습니다.
2025-07-24 12:25:43,685 - INFO - ✅ 테이블 'policy_categories'에 5602개의 행을 성공적으로 적재했습니다.
2025-07-24 12:25:43,800 - INFO - ✅ 테이블 'policy_subcategories'에 5602개의 행을 성공적으로 적재했습니다.
2025-07-24 12:25:43,801 - INFO - 🎉 모든 데이터 적재 작업이 성공적으로 완료되었습니다!
